# Natural Language Processing with Disaster Tweets

With the ever increasing role of Twitter in modern communications, multiple organizations are interested in programatically monitoring Twitter to detect occurring emergencies. However, it is not always clear whether a message containing "alarming" words is about a real emergency or is only a figure of speech. The goal of this project is to create a Regression Network model capable of predicting which Tweets are about real disasters and which one’s aren’t. 

For any given image, a positive label '1' indicates that the Tweet is about a real disaster, with a negative label '0' otherwise. The dataset consists of 10,000 tweets that were hand classified by the company "Figure-Eight".

## Setup

### Exploratory Data Analysis

### Dataset Construction

## Model Architecture

### Model Selection

#### Model Training

#### Model Results

## Alternative Models

### Hyperparameter tuning

### Alternative Model Architecture


## Results

### Model comparison results

### Hyper-parameter optimization results


## Conclusions

### Learnings

### Future works

## References

* [Addison Howard, devrishi, Phil Culliton, Yufeng Guo. (2019). Natural Language Processing with Disaster Tweets. Kaggle.](https://kaggle.com/competitions/nlp-getting-started)